In [1]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import math as m

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-14 10:39:00.668899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def gen_series():
  i = 0
  while True:
    size = np.random.randint(0, 10)
    yield i, np.random.normal(size=(size,))
    i += 1


In [3]:
ds_series = tf.data.Dataset.from_generator(
    gen_series, 
    output_types=(tf.int32, tf.float32), 
    output_shapes=((), (None,)))

ds_series

2023-04-14 10:39:05.723084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7494 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


<_FlatMapDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [4]:
q = next(iter(ds_series))
q

2023-04-14 10:39:09.481111: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.87242204,  0.46126366,  0.3283314 , -0.06489328, -1.1206204 ,
        -0.26687726, -0.3141946 ], dtype=float32)>)

In [5]:
dataset = load_dataset("speech_commands", "v0.01", split="train")

Found cached dataset speech_commands (/root/.cache/huggingface/datasets/speech_commands/v0.01/0.1.0/ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da)


In [7]:
def ds_gen():
    for record in dataset:
        x = record["audio"]["array"]
        y = record["label"]
        x_len = x.shape[0]

        mean = np.mean(x)
        std = np.std(x)
        x = (x - mean)/std

        if 10 <= y < 30:
            y = 10 #set others
        if y == 30:
            y = 11 #set silence

        if x_len > 16000:
            xs = np.array_split(x, m.ceil(x_len/16000))
        else:
            xs = [x]

        for a in xs:
            if a.shape[0] < 16000:
                yield (np.append(a, [0]*(16000 - a.shape[0])), y)
            else:
                yield (a, y)

In [8]:
tf_dataset = tf.data.Dataset.from_generator(ds_gen, output_signature=(
    tf.TensorSpec(shape=(16000,), dtype=tf.float64),
    tf.TensorSpec(shape=(), dtype=tf.int32)))